# Import, Cleaning & Merge datasets

In [2]:
import tqdm as notebook_tqdm

In [3]:
from dask.distributed import Client
import dask.dataframe as dd
import pandas as pd
from huggingface_hub import hf_hub_download
import matplotlib.pyplot as plt

# Исходные файлы загружены на ресурс https://huggingface.co/datasets
REPO_ID = "DmitriiMiptEdu/event_data"
adv = 'adv_data.csv'
event = 'event_data.csv'
users = 'users_data.csv'
# Start a Dask client
client = Client()

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
# event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  blocksize=25e6, dtype={'event_status': 'object','user_phone': 'object'})
event_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=event, repo_type="dataset"), low_memory=False,  dtype={'event_status': 'object','user_phone': 'object'})
event_data['user_phone'] = event_data['user_phone'].str.replace('+', '', regex=False)
# Преобразование полей датасетов во временные метки
event_data['event_dtm'] = dd.to_datetime(event_data['event_dtm'])
event_data['update_dtm'] = dd.to_datetime(event_data['update_dtm'])

In [12]:
event_data.compute()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
0,1263769bddda6eaaf7c8b8b22812ad6a063ad234d80782...,79140092454,show,2023-10-05 12:03:04.125806,NaN,NaT,NaN,RUB,mortgage
1,ae05865023d28ae72dd17955851263ead2d4daf11c69e0...,79125904946,show,2023-05-16 10:16:54.554523,NaN,NaT,NaN,RUB,loan
2,9861b00e9c9a99a6fdb2e5c11beebe9581eb1dda36ddc4...,79184556300,show,2023-12-23 22:46:44.066599,NaN,NaT,NaN,RUB,loan
3,349366af4823add107077299482f21031941fc6d4cd571...,79119287528,show,2023-07-30 05:25:46.285514,NaN,NaT,NaN,RUB,loan
4,2be2933460f58bb2d9a311ede6bb6ee064989568ab57b8...,79119287528,click,2023-07-30 05:25:46.285514,NaN,2023-07-30 05:42:59.014970,2.18,USD,loan
...,...,...,...,...,...,...,...,...,...
499968,5a8fc7ddcf726cd7abf21b6d27a76faa20abb9e74fb3cc...,79127118227,show,2023-11-14 15:48:21.377362,NaN,NaT,NaN,RUB,mortgage
499969,e6b7fa236dfd3d31983d81552eed72bac322aacd6517fe...,79124063664,show,2023-05-13 07:32:01.424318,NaN,NaT,NaN,RUB,loan
499970,73fa468e8552c3d27b123256e3c583ebe33472534d732d...,79170525726,show,2023-06-30 18:28:49.295919,NaN,NaT,NaN,RUB,mortgage
499971,a08a6b298ec487c02ca3466112a494954f688e3f0669e2...,79143941135,show,2023-07-07 08:58:40.684829,NaN,NaT,NaN,RUB,mortgage


In [11]:
event_data.loc

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency,product_type
npartitions=20,,,,,,,,,
,string,string,string,datetime64[ns],string,datetime64[ns],float64,string,string
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [4]:
users_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=users, repo_type="dataset"),low_memory=False,  dtype={'registration_dtm': 'object', 'delete_dtm': 'object','user_phone': 'object'} )
users_data['user_phone'] = users_data['user_phone'].str.replace('+', '', regex=False)
# Преобразование полей датасетов во временные метки
users_data['registration_dtm'] = dd.to_datetime(users_data['registration_dtm'])
users_data['delete_dtm'] = dd.to_datetime(users_data['delete_dtm'])

In [12]:
users_data.set_index('user_id')

In [13]:
users_data.compute()

,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,NaT
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,NaT
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,NaT
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,NaT
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,NaT
...,...,...,...,...
922973,79199999934,NaN,2023-10-09 11:11:54.760893,NaT
922974,79199999942,NaN,2023-09-03 18:18:01.703872,NaT
922975,79199999946,NaN,2023-06-24 14:15:41.180750,NaT
922976,79199999985,NaN,2023-02-26 22:46:23.643170,NaT


In [15]:
users_data.head()

,user_phone,user_id,registration_dtm,delete_dtm
0,79100004508,0000038524a303ed5876b823574848e0,2023-01-15 15:33:14.952938,NaT
1,79100005383,000003ef5bdfb2545c8d5ab693142e30,2022-11-21 22:43:54.418792,NaT
2,79100006444,0000056caffa28b60bf17bc69fe4366f,2022-12-23 03:00:04.088219,NaT
3,79100007221,00000a6f236f49c556e9a7b31d1b01e4,2022-12-23 03:31:50.935786,NaT
4,79100009619,00000cbd32277c99f2bd5e180a21f2b2,2023-01-25 14:31:17.234998,NaT


In [7]:
merge_data = event_data.set_index('user_phone').join(users_data.set_index('user_phone'))

In [ ]:
merge_data.compute()

In [7]:
adv_data = dd.read_csv(hf_hub_download(repo_id=REPO_ID, filename=adv, repo_type="dataset"), low_memory=False)
# Преобразование полей датасетов во временные метки
adv_data['campaign_start_dtm'] = dd.to_datetime(adv_data['campaign_start_dtm'])
adv_data['campaign_end_dtm'] = dd.to_datetime(adv_data['campaign_end_dtm'])

In [9]:
adv_data.set_index('user_id')

,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
npartitions=21,,,,,,,,
0000038524a303ed5876b823574848e0,string,datetime64[ns],datetime64[ns],string,string,string,string,float64
0cb65120ae8a5200b1c9b59ca566830d,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
f33d6c74abf88bb8c689d8504ca799b3,...,...,...,...,...,...,...,...
fffffdf2c04b4d26fa3463983dc73eb5,...,...,...,...,...,...,...,...


In [10]:
adv_data.info()

<class 'dask_expr.DataFrame'>
Columns: 9 entries, user_id to campaign_cost
dtypes: datetime64[ns](2), float64(1), string(6)

In [14]:
adv_data.compute()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,NaN
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,NaN
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,NaN
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,NaN
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34
...,...,...,...,...,...,...,...,...,...
476123,8d55b3f02b391d60043be246af1a5937,show,2023-10-12 21:35:47.522044,2023-10-12 21:35:47.522044,google / organic,reels_adv_retarget,api,RUB,NaN
476124,d9e3430962a672bdf2c05ad7f5043753,show,2023-03-29 07:07:42.068660,2023-03-29 07:07:42.068660,android / mobile_app,megamaket_case_bk,app,RUB,NaN
476125,f705d14d2af61158ef2d481878b1a2e8,show,2023-05-11 02:24:29.186321,2023-05-11 02:24:29.186321,google / organic,rebrand_title_target,web,RUB,NaN
476126,c5ab9d4afe732c6a6d4f37a6beb3e32b,show,2023-06-26 19:26:36.935691,2023-06-26 19:26:36.935691,vk / cpc,adv_reels_megamaket,app,RUB,NaN


In [16]:
adv_data.head()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost
0,db00f9c699edacc50090172cfaa3b43a,show,2023-09-18 19:39:25.390618,2023-09-18 19:39:25.390618,finance.rambler.ru / cpm,rebrand_megamaket_reels,app,RUB,NaN
1,88aa22bca4fa1d6da73deb4b5c5ee8c0,show,2023-04-11 06:47:33.199369,2023-04-11 06:47:33.199369,yandex / organic,adv_retarget_brandkeys,web,RUB,NaN
2,e9e12347e50732804d2f8ded00743a07,show,2023-10-28 04:35:48.329870,2023-10-28 04:35:48.329870,yandex / organic,bk_light_brandkeys,web,RUB,NaN
3,fa0cd363d1978a27a078152cd263c670,show,2023-05-28 18:25:18.694051,2023-05-28 18:25:18.694051,yandex / organic,retarget_mind_response,api,RUB,NaN
4,fa0cd363d1978a27a078152cd263c670,click,2023-05-31 18:55:55.352680,2023-05-31 18:55:55.352680,regular / sms,light_clickarrow_clickarrow,api,USD,1.34


In [18]:
adv_merge = dd.merge(adv_data, users_data, on ='user_id', how = 'left')

In [19]:
adv_merge.compute()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency,campaign_cost,user_phone,registration_dtm,delete_dtm
0,2b9c66fab937ab39bdeaf0e6368c654f,show,2023-04-22 18:45:41.313746,2023-04-22 18:45:41.313746,google / organic,megamaket_megamaket_client,app,RUB,NaN,79116442326,2022-12-17 05:31:55.462996,NaT
1,20957a29fcb374b9ef5e8e69afcde35c,show,2022-12-08 10:10:46.433374,2022-12-08 10:10:46.433374,google / organic,reels_brandkeys_brandkeys,app,RUB,NaN,79112111960,2022-11-17 16:59:41.477808,NaT
2,cda7357c2e116a80eec70a6a716040ba,show,2023-05-11 05:47:51.087815,2023-05-11 05:47:51.087815,yandex / organic,rebrand_brandkeys_mind,app,RUB,NaN,79180175222,2022-10-06 19:14:51.636451,NaT
3,7ef9f1f17b40bfc3d0a460ff04c77a1e,show,2023-12-22 16:06:58.250082,2023-12-22 16:06:58.250082,trigger / sms,rebrand_target_megamaket,app,RUB,NaN,79149229725,2023-01-07 00:25:54.428941,NaT
4,234006544877e0cf44e6dfbc9d48c728,show,2023-05-14 14:44:53.861134,2023-05-14 14:44:53.861134,yandex / cpc,target_reels_adv,app,RUB,NaN,79113161390,2023-04-23 13:41:09.344602,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
475370,4432f46bda26e6cec2ce0308adb1cfbc,click,2023-07-06 08:08:06.417761,2023-07-06 08:08:06.417761,yandex / cpc,megamaket_rebrand_bk,app,RUB,0.0,79126134075,2022-12-14 16:27:48.393745,NaT
475371,53a9af6c3010342f0b9bee013623fb3b,show,2023-11-28 01:39:51.469907,2023-11-28 01:39:51.469907,yandex / cpc,megamaket_megamaket_title,api,RUB,NaN,79132223395,2022-05-13 02:33:56.015710,NaT
475372,1c3bf849393883f0d2c55da9ca753f66,show,2023-03-24 01:20:33.855973,2023-03-24 01:20:33.855973,not-set / none,NaN,web,USD,NaN,79110408628,2022-10-06 22:03:22.613978,NaT
475373,e945bfa264389279dd764e8968fc55f6,show,2023-05-25 13:21:18.807508,2023-05-25 13:21:18.807508,google / organic,brandkeys_response_clickarrow,web,RUB,NaN,79191046724,2023-04-04 11:15:45.841990,NaT


In [27]:
del(adv_event_merge)

In [28]:
adv_event_merge = dd.merge(adv_merge, event_data, on ='user_phone', how = 'inner')

In [30]:
adv_event_merge = adv_event_merge.persist()

In [31]:
adv_event_merge.compute()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_x,campaign_cost,user_phone,registration_dtm,delete_dtm,id,event_type,event_dtm,event_status,update_dtm,payout,currency_y,product_type
0,9cb94c384ac97b954f53d463acea562c,show,2023-03-28 12:40:23.495769,2023-03-28 12:40:23.495769,google / organic,light_adv_adv,web,RUB,NaN,79160923407,2022-06-13 10:10:04.167935,NaT,76bd2dd812c03bf2faa688efe5db4779c2297852069726...,show,2023-02-11 10:46:59.432024,NaN,NaT,NaN,RUB,deposits
1,5980c8b2f4fa1158aaa15500122bd287,show,2023-07-26 13:45:23.217136,2023-07-26 13:45:23.217136,email / email,reels_bk_operator,web,RUB,NaN,79134516052,2023-01-17 01:25:37.377625,NaT,e2ae4edbccdbbf6f346d72facc0393e632fe684ebdb14c...,show,2023-08-11 12:46:21.669500,NaN,NaT,NaN,USD,loan
2,941ad8531f40a191219859e681527d46,show,2023-04-24 22:25:03.550215,2023-04-24 22:25:03.550215,email / email,bk_reels_case,app,RUB,NaN,79157539880,2022-10-28 10:14:50.214622,NaT,a280e131650c30eef588a969f1af8313c03fda445661b8...,show,2023-07-06 18:13:06.282192,NaN,NaT,NaN,RUB,mortgage
3,fe8ba40fc517db957dc62d8c5fd7019b,show,2023-09-01 17:14:01.113352,2023-09-01 17:14:01.113352,android / mobile_app,reels_response_case,app,USD,NaN,79199429922,2023-05-27 22:08:24.303154,NaT,c64f8f215de0d9c59acc8daea61a8b2e8a4b959ce5f8e6...,show,2023-06-11 00:10:39.727913,NaN,NaT,NaN,RUB,deposits
4,37bb75848c9359628f2bd4536bc4d66f,show,2023-11-30 17:54:59.843964,2023-11-30 17:54:59.843964,android / mobile_app,market_bk_case,app,RUB,NaN,79121206892,2023-05-31 01:00:47.758223,NaT,e82340729a414ba7afa36157648721334e2da967aa414b...,show,2023-09-23 00:11:58.563525,NaN,NaT,NaN,USD,mortgage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642793,5e9eb82e9fc8daf13468f06b5a02780d,show,2023-12-28 07:47:12.965541,2023-12-28 07:47:12.965541,google / organic,operator_reels_market,web,RUB,NaN,79136528817,2023-06-27 21:43:55.035844,NaT,fd08a025d4f32d26cd11ed5818a781fa59d6fa6c676216...,show,2023-09-16 08:06:54.395052,NaN,NaT,NaN,RUB,loan
642794,5e9eb82e9fc8daf13468f06b5a02780d,show,2023-12-28 07:47:12.965541,2023-12-28 07:47:12.965541,google / organic,operator_reels_market,web,RUB,NaN,79136528817,2023-06-27 21:43:55.035844,NaT,06f98517146bba9a598c4d09b2a3b056de6f189ba3f08c...,click,2023-09-16 08:06:54.395052,NaN,2023-09-16 08:18:34.429653,0.0,RUB,loan
642795,2f26947842992cb440e87a03a712e152,show,2023-07-15 08:03:55.770115,2023-07-15 08:03:55.770115,ios / mobile_app,title_adv_client,app,RUB,NaN,79117825982,2023-02-08 23:02:05.818267,NaT,180c8ee4af6c37eee7099d199a46b2a65d8c99437d0009...,show,2023-06-11 07:52:11.608502,NaN,NaT,NaN,RUB,deposits
642796,16fecdaa077147c2a0266c658b30e7da,show,2023-10-29 13:19:00.449780,2023-10-29 13:19:00.449780,yandex / organic,title_bk_mind,web,RUB,NaN,79108369246,2023-02-06 03:10:53.159065,NaT,f6f28de1df12d38b7d5557c4bd4b3a6d6c9f72c273540f...,show,2023-03-14 01:26:02.150539,NaN,NaT,NaN,RUB,loan


In [26]:
adv_event_merge.compute()

,user_id,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_x,campaign_cost,user_phone,registration_dtm,delete_dtm,id,event_type,event_dtm,event_status,update_dtm,payout,currency_y,product_type
0,cdfc862fc874ecbb9ae3a209453ff1b6,show,2023-03-01 01:03:51.500681,2023-03-01 01:03:51.500681,yandex / organic,mind_bk_response,api,RUB,NaN,79180307132,2022-12-20 16:37:24.937908,NaT,c13e3cbddc756a1c040e4820114e1e03adb4868174b92d...,show,2024-01-12 06:57:30.635756,NaN,NaT,NaN,RUB,loan
1,cdfc862fc874ecbb9ae3a209453ff1b6,show,2023-03-01 01:03:51.500681,2023-03-01 01:03:51.500681,yandex / organic,mind_bk_response,api,RUB,NaN,79180307132,2022-12-20 16:37:24.937908,NaT,0dd2544426a8656d7ec766eda95dec89aa186ac1c3ad82...,click,2024-01-12 06:57:30.635756,NaN,2024-01-12 07:04:14.835837,0.0,RUB,loan
2,6eccbd2ba527f2b8b8085afbcc07a24c,show,2023-11-20 14:44:59.742701,2023-11-20 14:44:59.742701,ya.ru / cpc,brandkeys_client_case,web,RUB,NaN,79142865138,2023-03-19 22:15:30.355189,NaT,e219b25428c31e04a1d58eb86afcdb35b8df5f08c2e237...,show,2023-08-14 05:39:45.281835,NaN,NaT,NaN,RUB,loan
3,6eccbd2ba527f2b8b8085afbcc07a24c,click,2023-10-09 12:04:54.229455,2023-10-09 12:04:54.229455,yandex / organic,response_target_case,web,RUB,0.0,79142865138,2023-03-19 22:15:30.355189,NaT,e219b25428c31e04a1d58eb86afcdb35b8df5f08c2e237...,show,2023-08-14 05:39:45.281835,NaN,NaT,NaN,RUB,loan
4,c5b720856c25827545456e8850c68964,show,2023-07-10 20:13:29.887971,2023-07-10 20:13:29.887971,not-set / none,NaN,web,USD,NaN,79177060453,2023-02-28 05:48:26.523494,NaT,a67f535e1e7ea5e21a739959d4da72a3fcb5bf7e45c472...,show,2023-07-11 17:48:54.029838,NaN,NaT,NaN,RUB,loan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642793,449860f3e93774f49788a36b8a2d9938,show,2023-04-11 21:13:53.168500,2023-04-11 21:13:53.168500,regular / sms,reels_operator_bk,app,RUB,NaN,79126291783,2022-12-09 00:50:09.539160,NaT,fe05c972a7708a7c79db3492dfe872c2daab21aea996f9...,show,2023-03-12 07:46:13.063958,NaN,NaT,NaN,RUB,loan
642794,449860f3e93774f49788a36b8a2d9938,show,2023-04-11 21:13:53.168500,2023-04-11 21:13:53.168500,regular / sms,reels_operator_bk,app,RUB,NaN,79126291783,2022-12-09 00:50:09.539160,NaT,8f91912ebba614aef7e6c0f9f46fa9d68d7e924d86f659...,click,2023-03-12 07:46:13.063958,NaN,2023-03-12 07:56:18.217865,0.0,RUB,loan
642795,2261c75e24116eaf6179e57fc81739c6,show,2023-03-25 03:48:31.994280,2023-03-25 03:48:31.994280,yandex / organic,market_retarget_client,web,RUB,NaN,79112819742,2023-03-06 13:25:35.788386,NaT,05c8db27ec97554c2e0e1869ec69c9683db59b9b807b7b...,show,2023-10-01 08:52:24.141714,NaN,NaT,NaN,RUB,loan
642796,ae994d49d8786643c4389ab71d1b2eca,show,2023-12-13 02:51:14.166609,2023-12-13 02:51:14.166609,google / organic,case_retarget_adv,web,RUB,NaN,79167965446,2023-06-21 20:56:35.376735,NaT,1b491d3143bb51c8bf71e41e69d1d9ce56d48d9d6817f5...,show,2023-07-07 09:27:20.094544,NaN,NaT,NaN,RUB,loan


In [33]:
# Объединение users_data и event_data
merged_data = event_data.merge(users_data, on='user_phone', how='right').compute()

In [34]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10009731 entries, 0 to 499935
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   id                object        
 1   user_phone        object        
 2   event_type        object        
 3   event_dtm         datetime64[ns]
 4   event_status      object        
 5   update_dtm        datetime64[ns]
 6   payout            float64       
 7   currency          object        
 8   product_type      object        
 9   user_id           object        
 10  registration_dtm  datetime64[ns]
 11  delete_dtm        datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(7)
memory usage: 992.8+ MB


In [16]:
merged_data.head(10)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,registration_dtm,delete_dtm,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost
4296567,f3d70cc9c1a0e6024bb8a6932d6cf13cc087818a49e470...,79100000015,show,2023-04-21 04:25:03.775939,NaN,NaT,NaN,RUB,loan,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3433033,6c07ec4d6011e7585d721e7161b353727ee7a7ef91b36a...,79100000044,show,2023-03-27 00:46:11.341816,NaN,NaT,NaN,RUB,loan,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3433034,b19665e8baa9cb4380cedb19e5f7fd4a133dfe219d5f72...,79100000044,click,2023-03-27 00:46:11.341816,NaN,2023-03-27 00:57:18.382909,0.0,RUB,loan,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1509817,b36b86fcdf9fa1bf44d9757a2d762106379e90a4873c85...,79100000064,show,2023-12-02 04:28:23.773632,NaN,NaT,NaN,RUB,loan,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
1509818,38da8550dd1a284ceb27775e28ecf19cc2c08a419e50e8...,79100000064,click,2023-12-02 04:28:23.773632,NaN,2023-12-02 04:43:18.888458,0.0,RUB,loan,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
6755748,98faaeafa6781bc157295f679a98e983918832fc45668a...,79100000069,show,2023-04-14 12:38:04.142221,NaN,NaT,NaN,RUB,mortgage,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3465232,1800e7b8be336d9d7fd40a7135801733b69dacc79428c4...,79100000083,show,2023-10-03 02:57:48.845910,NaN,NaT,NaN,RUB,mortgage,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
7028887,4b2e9e657d6ef00d928748480397e350aa3c04f31046ff...,79100000088,show,2023-03-15 18:18:22.601843,NaN,NaT,NaN,RUB,deposits,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
3662251,2f20d1af8ec0943399a6fa75b0de8bd7e5c3900e21ab0d...,79100000093,show,2023-12-08 04:38:51.680130,NaN,NaT,NaN,RUB,deposits,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN
7227635,98552848796ff55c715deccd9b16fcacecb0a2377d537d...,79100000107,show,2023-12-21 04:19:59.675974,NaN,NaT,NaN,RUB,mortgage,NaN,NaT,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN


In [8]:

# Объединение с adv_data
merged_data = dd.merge(merged_data, adv_data, on='user_id', how='left')

In [34]:
merged_data.head(10)

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,registration_dtm,delete_dtm,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost
0,db2d3fb233f1a9dc8076bce0f7f7eb500a68f2e4328f05...,+79162021850,show,2024-01-03 21:10:11.133388,NaN,NaT,NaN,RUB,loan,9f839cfdf98473552f7d54be71817dea,2023-05-01 14:33:20.049524,NaT,show,2023-07-30 17:52:54.059633,2023-07-30 17:52:54.059633,yandex / organic,target_client_rebrand,app,RUB,NaN
1,1ed7b0f97e76477ac4920fe9f4f255e006c89228686690...,+79162021850,click,2024-01-03 21:10:11.133388,NaN,2024-01-03 21:17:48.178325,0.0,RUB,loan,9f839cfdf98473552f7d54be71817dea,2023-05-01 14:33:20.049524,NaT,show,2023-07-30 17:52:54.059633,2023-07-30 17:52:54.059633,yandex / organic,target_client_rebrand,app,RUB,NaN
2,c91d37836d8a5884eeb3b7a1ea77b864a76ad1fedd0591...,+79148227089,show,2023-02-28 03:34:31.492589,NaN,NaT,NaN,RUB,mortgage,7c6a806f145ba76cdcb1fec6b3b202f3,2022-11-18 12:36:05.213179,NaT,show,2023-06-21 10:21:50.755443,2023-06-21 10:21:50.755443,ios / mobile_app,light_market_adv,app,RUB,NaN
3,e9fdb7d5b88253427e159579c53e080c7f8a27f5f6b02e...,+79157796431,show,2023-04-16 06:26:18.296362,NaN,NaT,NaN,RUB,loan,94c3497d3909ee256f6aa01005c6a21f,2022-12-06 13:20:25.968799,NaT,show,2023-09-28 11:42:50.921042,2023-09-28 11:42:50.921042,google / organic,clickarrow_adv_clickarrow,app,RUB,NaN
4,e9fdb7d5b88253427e159579c53e080c7f8a27f5f6b02e...,+79157796431,show,2023-04-16 06:26:18.296362,NaN,NaT,NaN,RUB,loan,94c3497d3909ee256f6aa01005c6a21f,2022-12-06 13:20:25.968799,NaT,click,2023-11-26 00:40:58.413915,2023-11-26 00:40:58.413915,yandex / organic,mind_bk_client,web,RUB,0.0
5,6a4de0e4553bab1a743e0df48f572105f7b520a5055a2f...,+79168865804,show,2023-03-01 06:22:43.940051,NaN,NaT,NaN,RUB,loan,b0dec61e1a92f2d728b4fb09096ed132,2022-08-29 00:55:03.467320,NaT,show,2022-12-13 19:43:10.802567,2022-12-13 19:43:10.802567,yandex / organic,market_retarget_case,app,RUB,NaN
6,6a4de0e4553bab1a743e0df48f572105f7b520a5055a2f...,+79168865804,show,2023-03-01 06:22:43.940051,NaN,NaT,NaN,RUB,loan,b0dec61e1a92f2d728b4fb09096ed132,2022-08-29 00:55:03.467320,NaT,click,2023-01-25 06:42:41.672197,2023-01-25 06:42:41.672197,yandex / cpc,light_case_reels,app,RUB,0.0
7,a7c42af8a80ce5b364bcd7963492caeacbe8faf2d9101a...,+79128114885,show,2023-07-13 07:47:54.033235,NaN,NaT,NaN,RUB,mortgage,493b8d37c80db36eec1a5b24e0d29170,2023-07-01 23:08:24.270742,NaT,show,2023-09-01 12:27:02.810640,2023-09-01 12:27:02.810640,google / organic,rebrand_market_bk,web,RUB,NaN
8,634460106c688ed12d9ce5508a9582cb411c9ef342c9cb...,+79100674773,show,2023-06-09 02:18:41.735490,NaN,NaT,NaN,RUB,loan,0371763d0902c11970bf60d8b422c873,2023-03-25 02:21:30.148443,NaT,show,2023-05-08 14:30:00.306183,2023-05-08 14:30:00.306183,yandex / cpc,light_client_mind,app,USD,NaN
9,634460106c688ed12d9ce5508a9582cb411c9ef342c9cb...,+79100674773,show,2023-06-09 02:18:41.735490,NaN,NaT,NaN,RUB,loan,0371763d0902c11970bf60d8b422c873,2023-03-25 02:21:30.148443,NaT,click,2023-05-11 18:37:58.644710,2023-05-11 18:37:58.644710,yandex / cpc,operator_client_title,api,RUB,0.0


In [9]:
# Анализ событий пользователей
# Сортировка событий по пользователям и времени
merged_data = merged_data.sort_values(by=['user_phone', 'event_dtm'])

In [10]:
# Идентификация конверсий
conversions = merged_data[merged_data['event_status'] == 'success']

In [11]:
# Преобразование в pandas DataFrame для дальнейшей обработки
merged_data_pd = conversions.compute()

In [15]:
merged_data_pd.head()

,id,user_phone,event_type,event_dtm,event_status,update_dtm,payout,currency_x,product_type,user_id,registration_dtm,delete_dtm,campaign_type,campaign_start_dtm,campaign_end_dtm,source_medium,utm_campaign,interface,currency_y,campaign_cost


# Разработка модели атрибуции

In [14]:
# Реализация модели линейной атрибуции
def linear_attribution(events):
    n = len(events)
    attribution = 1.0 / n
    events['attribution'] = attribution
    return events

# Применение линейной модели атрибуции
#attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(linear_attribution).reset_index(drop=True)
attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(lambda x: linear_attribution(x)).reset_index(drop=True)




/tmp/ipykernel_1450/3458240431.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  attributed_data = merged_data_pd.groupby('user_phone', group_keys=False).apply(lambda x: linear_attribution(x)).reset_index(drop=True)


In [44]:
attributed_data.head()

# Визуализация результатов

In [ ]:
# Сбор данных для визуализации
source_attribution = attributed_data.groupby('source_medium')['attribution'].sum().compute()
print(source_attribution)

# Визуализация результатов
source_attribution_pd = source_attribution.compute()

# Пример простой визуализации распределения атрибуции
source_attribution_pd.plot(kind='bar')
plt.title('Атрибуция по источникам трафика')
plt.xlabel('Источник')
plt.ylabel('Атрибуция')
plt.show()